In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/home/abhayrokkam/projects/chat-memory/.chat-memory/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Clearing cuda cache
torch.cuda.empty_cache()

# Models

In [3]:
judge_model_id = "microsoft/Phi-3-mini-4k-instruct"
# extract_model_id = "Qwen/Qwen2.5-7B-Instruct"

# A model to judge if there is relevant information in the given text
judge_model = AutoModelForCausalLM.from_pretrained( 
    judge_model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

judge_model_tokenizer = AutoTokenizer.from_pretrained(judge_model_id)

# # Extraction model to extract the relevant information
# extract_model = AutoModelForCausalLM.from_pretrained( 
#     extract_model_id,
#     device_map="cuda",
#     torch_dtype="auto",
#     trust_remote_code=True,
# )

# extract_model_tokenizer = AutoTokenizer.from_pretrained(extract_model_id)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.05s/it]


In [4]:
# Pipeline for the judgement model
judge_pipe = pipeline(
    "text-generation",
    model=judge_model,
    tokenizer=judge_model_tokenizer,
)

judge_generation_args = {
    "max_new_tokens": 1,
    "return_full_text": False,
    "temperature": 0.5,
    "do_sample": True,
}

In [5]:
# Pipeline for the extraction model
extract_pipe = pipeline(
    "text-generation",
    # model=extract_model,
    # tokenizer=extract_model_tokenizer,
    model=judge_model,
    tokenizer=judge_model_tokenizer,
)

extract_generation_args = {
    "max_new_tokens": 250,
    "return_full_text": False,
    "temperature": 0.8,
    "do_sample": True,
}

# User Input

In [14]:
user_content = "I was diagnosed with Arthritis"

# Judgement

In [7]:
judge_system_content = """
    Your duty is to assist the personal companion. You have to decide if the personal companion will benefit by remebering the facts from the given text.
    The personal companion is expected to remember everything important to an individual. The personal companion is a best friend who takes care of the User.
    The personal companion is a companion for old individuals.
    The personal companion is expected to remember family and related matters, milestones in life, health issues, medication schedules.

    If you think the text contains relevant information, you have to respond with 'TRUE'.
    If there is no relevant information in the text, return 'FALSE'.
    Do NOT say anything else. You have to respond ONLY with either 'TRUE' or 'FALSE'.
    
    You are the Assistant in the given examples.
    
    Examples:
    
    User: Hey, my name is Adam.
    Assistant: TRUE
    
    User: I love my brother John.
    Assistant: TRUE
    
    User: The sky is blue today.
    Assistant: FALSE
"""

In [15]:
judge_messages = [ 
    {"role": "system", "content": judge_system_content},
    {"role": "user", "content": user_content},
]

In [16]:
output = judge_pipe(judge_messages, **judge_generation_args)
judgement = output[0]['generated_text']
print(judgement)

 TRUE


# Extraction

In [10]:
extract_system_content = """
    Your duty is to assist the personal companion. You have to extract the facts from the given text that will benefit hte personal companion.
    The personal companion is expected to remember everything important to an individual. The personal companion is a best friend who takes care of the User.
    The personal companion is a companion for old individuals.
    The personal companion is expected to remember family and related matters, milestones in life, health issues, medication schedules.
    
    You have to extract the relevant information from the given text.
    
    You are the Assistant in the given examples.
    
    Examples:
    
    User: Hey, my name is Adam.
    Assistant: Name is Adam.
    
    User: I love my brother John.
    Assistant: Brother named John.
    
    User: The sky is blue today.
    Assistant: 
    
    User: I love my wife Emily and we just came from Coldplay concert. I love them.
    Assistant: Wife named Emily. Likes Coldplay.
"""

In [17]:
extract_messages = [ 
    {"role": "system", "content": extract_system_content},
    {"role": "user", "content": user_content},
]

In [18]:
if judgement == ' TRUE':
    output = extract_pipe(extract_messages, **extract_generation_args)
    memory = output[0]['generated_text']
    print(memory)

 Diagnosed with Arthritis.
